In [1]:
import glob
import pickle
import numpy
from music21 import converter, instrument, note, chord, stream
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.layers import Activation
from keras.utils import np_utils
from keras.callbacks import ModelCheckpoint

Fonction général appelant les autres fonctions pour générer la musique en format MIDI

In [2]:
def generate():
    
    # Chargement des notes utilisées pour entrainer le modèle
    with open('/content/drive/MyDrive/data/notes', 'rb') as filepath:
        notes = pickle.load(filepath)


    pitchnames = sorted(set(item for item in notes))
 
    n = len(set(notes))

    network_input, normalized_input = prepare_sequences(notes, pitchnames, n)
    model = create_network(normalized_input, n)
    prediction_output = generate_notes(model, network_input, pitchnames, n)
    create_midi(prediction_output)

In [3]:
def prepare_sequences(notes, pitchnames, n_vocab):
    
    # Préparation des séquences utilisées par le réseau neuronal
    
    # Mapping invers d'entiers vers des notes
    note_to_int = dict((note, number) for number, note in enumerate(pitchnames))

    sequence_length = 100
    network_input = []
    output = []
    for i in range(0, len(notes) - sequence_length, 1):
        sequence_in = notes[i:i + sequence_length]
        sequence_out = notes[i + sequence_length]
        network_input.append([note_to_int[char] for char in sequence_in])
        output.append(note_to_int[sequence_out])

    n_patterns = len(network_input)

    # On utilise numpy pour mettre l'input dans un format compatible avec LSTM
    normalized_input = numpy.reshape(network_input, (n_patterns, sequence_length, 1))
  
    normalized_input = normalized_input / float(n_vocab)

    return (network_input, normalized_input)

In [4]:
# from google.colab import drive
# drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
def create_network(network_input, n):
    
    # Création de la structure du réseau neuronal
    
    model = Sequential()
    model.add(LSTM(
        512,
        input_shape=(network_input.shape[1], network_input.shape[2]),
        return_sequences=True
    ))
    model.add(Dropout(0.3))
    model.add(LSTM(512, return_sequences=True))
    model.add(Dropout(0.3))
    model.add(LSTM(512))
    model.add(Dense(256))
    model.add(Dropout(0.3))
    model.add(Dense(n))
    model.add(Activation('softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='rmsprop')

    # Chargement des poids des noeuds
    model.load_weights('/content/drive/MyDrive/data/weights-improvement-09-2.5811-bigger.hdf5')

    return model

In [6]:
def generate_notes(model, network_input, pitchnames, n_vocab):
    
    # Génération des notes
    
    # Séléction aléatoire d'une séquence comme début
    start = numpy.random.randint(0, len(network_input)-1)

    int_to_note = dict((number, note) for number, note in enumerate(pitchnames))

    pattern = network_input[start]
    prediction_output = []

    # Génération de 500 notes
    for note_index in range(500):
        prediction_input = numpy.reshape(pattern, (1, len(pattern), 1))
        prediction_input = prediction_input / float(n_vocab)

        prediction = model.predict(prediction_input, verbose=0)

        index = numpy.argmax(prediction) 
        result = int_to_note[index] 
        prediction_output.append(result) 

        pattern.append(index)
        pattern = pattern[1:len(pattern)]

    return prediction_output

In [7]:
def create_midi(prediction_output):
    
    # Convertit la sortie de la prédiction en notes et crée un fichier midi à partir des notes
    
    offset = 0
    output_notes = []

    # Crée des notes et chord basé sur les valeurs générées par le modèle
    for pattern in prediction_output:
        
        if ('.' in pattern) or pattern.isdigit():
            notes_in_chord = pattern.split('.')
            notes = []
            for current_note in notes_in_chord:
                new_note = note.Note(int(current_note))
                new_note.storedInstrument = instrument.SnareDrum()
                notes.append(new_note)
            new_chord = chord.Chord(notes)
            new_chord.offset = offset
            output_notes.append(new_chord)
        
        else:
            new_note = note.Note(pattern)
            new_note.offset = offset
            new_note.storedInstrument = instrument.SnareDrum()
            output_notes.append(new_note)

     
        offset += 0.5

    midi_stream = stream.Stream(output_notes)

    midi_stream.write('midi', fp='test_output3.mid')

In [ ]:
generate()